# Word2Vec

This notebook is a cleaned version of the Word2Vect practical, adapted to a French dataset, in order to get usable embeddings

In [2]:
import collections

import math
import numpy as np

import random
import sys
import time
import zipfile

In [3]:
## colab SETUP
!mkdir data
%cd data
!wget https://raw.githubusercontent.com/tomsercu/lstm/master/data/ptb.train.txt
ROOT_DIR='content'

import os
from pathlib import Path

ROOT_DIR = Path.home()

/home/quarknova/Desktop/NLProj/data
--2021-01-16 11:56:08--  https://raw.githubusercontent.com/tomsercu/lstm/master/data/ptb.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.120.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.120.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5101618 (4,9M) [text/plain]
Saving to: ‘ptb.train.txt’

ptb.train.txt       100%[===================>]   4,87M  2,04MB/s    in 2,4s    

2021-01-16 11:56:11 (2,04 MB/s) - ‘ptb.train.txt’ saved [5101618/5101618]



In [6]:
data_path = os.path.join(ROOT_DIR,'data/')
file = 'ptb.train.txt'
with open(data_path+file, 'r') as f:
    lines = f.readlines()
    # st is the abbreviation of "sentence" in the loop
    raw_dataset = [st.split() for st in lines]

'# sentences: %d' % len(raw_dataset)

'# sentences: 42068'

For the first three sentences of the data set, print the number of words and the first five words of each sentence. The end character of this data set is "&lt;eos&gt;", uncommon words are all represented by "&lt;unk&gt;", and numbers are replaced with "N".

In [7]:
for i in range(3):
    print(len(raw_dataset[i]))
    for j in range(5):
        print(raw_dataset[i][j])

24
aer
banknote
berlitz
calloway
centrust
15
pierre
<unk>
N
years
old
11
mr.
<unk>
is
chairman
of


### Create Word Index

For the sake of simplicity, we only keep words that appear at least 5 times in the data set.

In [8]:
# tk is an abbreviation for "token" in the loop
counter = collections.Counter([tk for st in raw_dataset for tk in st])
counter = dict(filter(lambda x: x[1] >= 5, counter.items()))

In [9]:
idx_to_token = [tk for tk, _ in counter.items()]
token_to_idx = {tk: idx for idx, tk in enumerate(idx_to_token)}
dataset = [[token_to_idx[tk] for tk in st if tk in token_to_idx]
           for st in raw_dataset]
num_tokens = sum([len(st) for st in dataset])
'# tokens: %d' % num_tokens

'# tokens: 887100'

In [10]:
idx_to_token[:5]

['pierre', '<unk>', 'N', 'years', 'old']

In [11]:
token_to_idx['consensus']

4827

### Subsampling

In text data, there are generally some words that appear at high frequencies, such "the", "a", and "in" in English. Generally speaking, in a context window, it is better to train the word embedding model when a word (such as "chip") and a lower-frequency word (such as "microprocessor") appear at the same time, rather than when a word appears with a higher-frequency word (such as "the"). Therefore, when training the word embedding model, we can perform subsampling[2] on the words. Specifically, each indexed word $w_i$ in the data set will drop out at a certain probability. The dropout probability is given as:

$$ \mathbb{P}(w_i) = \max\left(1 - \sqrt{\frac{t}{f(w_i)}}, 0\right),$$

Here, $f(w_i)$ is the ratio of the instances of word $w_i$ to the total number of words in the data set, and the constant $t$ is a hyper-parameter (set to $10^{-4}$ in this experiment). As we can see, it is only possible to drop out the word $w_i$ in subsampling when $f(w_i) > t$. The higher the word's frequency, the higher its dropout probability.

In [12]:
def discard(idx):
    return random.uniform(0, 1) < 1 - math.sqrt(
        1e-4 / counter[idx_to_token[idx]] * num_tokens)

subsampled_dataset = [[tk for tk in st if not discard(tk)] for st in dataset]
'# tokens: %d' % sum([len(st) for st in subsampled_dataset])

'# tokens: 375392'

In [13]:
def compare_counts(token):
    return '# %s: before=%d, after=%d' % (token, sum(
        [st.count(token_to_idx[token]) for st in dataset]), sum(
        [st.count(token_to_idx[token]) for st in subsampled_dataset]))

compare_counts('the')

'# the: before=50770, after=2129'

In [14]:
compare_counts('join')

'# join: before=45, after=45'

### Extract Central Target Words and Context Words

We use words with a distance from the central target word not exceeding the context window size as the context words of the given center target word. The following definition function extracts all the central target words and their context words. It uniformly and randomly samples an integer to be used as the context window size between integer 1 and the `max_window_size` (maximum context window).

In [15]:
def get_centers_and_contexts(dataset, max_window_size):
    centers, contexts = [], []
    for st in dataset:
        # Each sentence needs at least 2 words to form a
        # "central target word - context word" pair
        if len(st) < 2:
            continue
        centers += st
        for center_i in range(len(st)):
            window_size = random.randint(1, max_window_size)
            indices = list(range(max(0, center_i - window_size),
                                 min(len(st), center_i + 1 + window_size)))
            # Exclude the central target word from the context words
            indices.remove(center_i)
            contexts.append([st[idx] for idx in indices])
    return centers, contexts

In [18]:
tiny_dataset = [list(range(7)), list(range(7, 10))]
print('dataset', tiny_dataset)
for center, context in zip(*get_centers_and_contexts(tiny_dataset, 2)):
    print('center', center, 'has contexts', context)

dataset [[0, 1, 2, 3, 4, 5, 6], [7, 8, 9]]
center 0 has contexts [1, 2]
center 1 has contexts [0, 2, 3]
center 2 has contexts [0, 1, 3, 4]
center 3 has contexts [2, 4]
center 4 has contexts [2, 3, 5, 6]
center 5 has contexts [4, 6]
center 6 has contexts [5]
center 7 has contexts [8]
center 8 has contexts [7, 9]
center 9 has contexts [8]


In the experiment, we set the maximum context window size to 5. The following extracts all the central target words and their context words in the data set.

In [19]:
all_centers, all_contexts = get_centers_and_contexts(subsampled_dataset, 5)

## Negative Sampling

We use negative sampling for approximate training. For a central and context word pair, we randomly sample $K$ noise words ($K=5$ in the experiment). According to the suggestion in the Word2vec paper, the noise word sampling probability $\mathbb{P}(w)$ is the ratio of the word frequency of $w$ to the total word frequency raised to the power of 0.75.

In [20]:
def get_negatives(all_contexts, sampling_weights, K):
    all_negatives, neg_candidates, i = [], [], 0
    population = list(range(len(sampling_weights)))
    for contexts in all_contexts:
        negatives = []
        while len(negatives) < len(contexts) * K:
            if i == len(neg_candidates):
                # An index of k words is randomly generated as noise words
                # based on the weight of each word (sampling_weights). For
                # efficient calculation, k can be set slightly larger
                i, neg_candidates = 0, random.choices(population, sampling_weights, k=int(1e5))
            neg, i = neg_candidates[i], i + 1
            # Noise words cannot be context words
            if neg not in set(contexts):
                negatives.append(neg)
        all_negatives.append(negatives)
    return all_negatives

sampling_weights = [counter[w]**0.75 for w in idx_to_token]
all_negatives = get_negatives(all_contexts, sampling_weights, 5)

In [21]:
all_negatives[0]

[4555,
 74,
 3081,
 4878,
 2819,
 3349,
 1132,
 6819,
 942,
 2266,
 7548,
 5992,
 1459,
 4530,
 3890,
 1104,
 179,
 10,
 1092,
 2117,
 5755,
 2419,
 2397,
 4945,
 3974]

In [22]:
all_contexts[0]

[3, 5, 6, 8, 11]

## Reading Data

We extracted all central target words `all_centers`, and the context words `all_contexts` and noise words `all_negatives` of each central target word from the data set. We will read them in random mini-batches.

In a mini-batch of data, the $i$-th example includes a central word and its corresponding $n_i$ context words and $m_i$ noise words. Since the context window size of each example may be different, the sum of context words and noise words, $n_i+m_i$, will be different. When constructing a mini-batch, we concatenate the context words and noise words of each example, and add 0s for padding until the length of the concatenations are the same, that is, the length of all concatenations is $\max_i n_i+m_i$(`max_len`). In order to avoid the effect of padding on the loss function calculation, we construct the mask variable `masks`, each element of which corresponds to an element in the concatenation of context and noise words, `contexts_negatives`. When an element in the variable `contexts_negatives` is a padding, the element in the mask variable `masks` at the same position will be 0. Otherwise, it takes the value 1. In order to distinguish between positive and negative examples, we also need to distinguish the context words from the noise words in the `contexts_negatives` variable. Based on the construction of the mask variable, we only need to create a label variable `labels` with the same shape as the `contexts_negatives` variable and set the elements corresponding to context words (positive examples) to 1, and the rest to 0.

Next, we will implement the mini-batch reading function `batchify`. Its mini-batch input `data` is a list whose length is the batch size, each element of which contains central target words `center`, context words `context`, and noise words `negative`. The mini-batch data returned by this function conforms to the format we need, for example, it includes the mask variable. We wrap this function in a `Dataset` module.

In [23]:
import torch
import torch.nn as nn

In [24]:
from torch.utils.data import Dataset, DataLoader

In [25]:
class PTB_dataset(Dataset):
    
    def __init__(self, all_centers, all_contexts, all_negatives):
        self.all_centers, self.all_contexts_negatives, self.all_masks, self.all_labels = self.batchify(list(zip(all_centers,all_contexts,all_negatives)))
        
    def __len__(self):
        return len(self.all_centers)
    
    def __getitem__(self,idx):
        return self.all_centers[idx], self.all_contexts_negatives[idx], self.all_masks[idx], self.all_labels[idx]
        
    def batchify(self,data):
        max_len = max(len(c) + len(n) for _, c, n in data)
        centers, contexts_negatives, masks, labels = [], [], [], []
        for center, context, negative in data:
            cur_len = len(context) + len(negative)
            centers += [center]
            contexts_negatives += [context + negative + [0] * (max_len - cur_len)]
            masks += [[1] * cur_len + [0] * (max_len - cur_len)]
            labels += [[1] * len(context) + [0] * (max_len - len(context))]
        return (torch.tensor(centers).view((-1, 1)), torch.tensor(np.array(contexts_negatives)),
            torch.tensor(np.array(masks)), torch.tensor(np.array(labels)))
        

In [26]:
ptbdata = PTB_dataset(all_centers, all_contexts, all_negatives)

In [27]:
ptbdata[1]

(tensor([3]),
 tensor([   0,    5,    6,    8, 3523,  144, 3099, 5207,   23,  442,  649, 2549,
         1537, 3815,  127, 6354, 4318,  212, 5063,   94,  378,   94, 3100, 4196,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

And here is our dataloader.

In [28]:
batch_size = 512

data_loader = DataLoader(ptbdata, batch_size, shuffle=True,
                              num_workers=4)
for batch in data_loader:
    for name, data in zip(['centers', 'contexts_negatives', 'masks',
                           'labels'], batch):
        print(name, 'shape:', data.shape)
    break

centers shape: torch.Size([512, 1])
contexts_negatives shape: torch.Size([512, 60])
masks shape: torch.Size([512, 60])
labels shape: torch.Size([512, 60])


## The Skip-Gram Model

You will implement the skip-gram model by using embedding layers and mini-batch multiplication [`torch.einsum`](https://pytorch.org/docs/stable/torch.html#torch.bmm). These methods are also often used to implement other natural language processing applications.

In [29]:
# taken from the spotlight library, see
# https://github.com/maciejkula/spotlight/blob/master/spotlight/layers.py
class ScaledEmbedding(nn.Embedding):
    """
    Embedding layer that initialises its values
    to using a normal variable scaled by the inverse
    of the emedding dimension.
    """
    def reset_parameters(self):
        """
        Initialize parameters.
        """
        self.weight.data.normal_(0, 1.0 / self.embedding_dim)
        if self.padding_idx is not None:
            self.weight.data[self.padding_idx].fill_(0)

### Skip-gram Model Forward Calculation

In forward calculation, the input of the skip-gram model contains the central target word index `center` and the concatenated context and noise word index `contexts_and_negatives`. In which, the `center` variable has the shape (batch size, 1), while the `contexts_and_negatives` variable has the shape (batch size, `max_len`). These two variables are first transformed from word indexes to word vectors by the word embedding layer, and then the output of shape (batch size, 1, `max_len`) is obtained by mini-batch multiplication. Each element in the output is the inner product of the central target word vector and the context word vector or noise word vector.

In [95]:
class Skip_gram(nn.Module):
    def __init__(self, input_dim, embed_size = 100):
        super(Skip_gram, self).__init__()
        self.input_dim = input_dim
        self.embed_size = embed_size
        self.central_emb = ScaledEmbedding(self.input_dim,self.embed_size)
        self.context_emb = ScaledEmbedding(self.input_dim,self.embed_size)
        
    def forward(self, icent, icont):
        # (hint: dimensions are for icent (bs,1) for icont (bs,max_len) and output (bs,1,max_len))
        output = torch.einsum('aij,akj->aik', self.central_emb(icent), self.context_emb(icont))
        return output

In [98]:
net = Skip_gram(len(idx_to_token))

In [99]:
net(torch.tensor([0,1,3]).unsqueeze(1),torch.tensor([[0,0],[0,0],[0,0]]))

tensor([[[ 0.0003,  0.0003]],

        [[-0.0011, -0.0011]],

        [[ 0.0006,  0.0006]]], grad_fn=<ViewBackward>)

### Loss function

It is worth mentioning that we need use the mask variable to specify the partial predicted value and label that participate in loss function calculation in the mini-batch: when the mask is 1, the predicted value and label of the corresponding position will participate in the calculation of the loss function; When the mask is 0, the predicted value and label of the corresponding position do not participate in the calculation of the loss function. As we mentioned earlier, mask variables can be used to avoid the effect of padding on loss function calculations.

In [163]:
loss_fn = nn.BCEWithLogitsLoss(reduction='none')
def criterion(pred, label, mask):
    #
    return [loss_fn(p, l.type_as(p)) * m for p, l, m in zip(pred, label, mask)]

In [164]:
pred = torch.tensor([[1.5, 0.3, -1, 2], [1.1, -0.6, 2.2, 0.4]])
# 1 and 0 in the label variables label represent context words and the noise
# words, respectively
label = torch.tensor([[1, 0, 0, 0], [1, 1, 0, 0]]).type(torch.FloatTensor)
mask = torch.tensor([[1, 1, 1, 1], [1, 1, 1, 0]]).type(torch.FloatTensor)  # Mask variable

criterion(pred,label,mask)

[tensor([0.2014, 0.8544, 0.3133, 2.1269]),
 tensor([0.2873, 1.0375, 2.3051, 0.0000])]

In [165]:
optimizer = torch.optim.Adam(net.parameters(),lr=0.005)

In [179]:
def train(n_epochs):
    
    for epoch in range(n_epochs):
        start, loss = time.time(), 0
        print(len(data_loader))
        for batch in data_loader:
            #
            cent, cont, mask, label = batch
            pred = torch.squeeze(net(cent, cont))
            loss = torch.sum(sum(criterion(pred, label, mask)))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        print('epoch %d, loss %.2f, time %.2fs'
              % (epoch + 1, loss, time.time() - start))

In [181]:
train(6)

732
epoch 1, loss 2001.61, time 177.00s
732
epoch 2, loss 1889.13, time 170.48s
732
epoch 3, loss 1886.85, time 180.14s
732
epoch 4, loss 1813.41, time 170.84s
732
epoch 5, loss 1725.01, time 166.89s
732
epoch 6, loss 1662.56, time 171.37s


In [205]:
def get_similar_tokens(query_token, k, W):
    #
    # your code here
    #
    token_id = token_to_idx[query_token]
    cos_sim = nn.CosineSimilarity()
    cos = torch.Tensor([cos_sim(net.central_emb(torch.LongTensor([token_id])), net.central_emb(torch.LongTensor([i]))) for i in range(len(idx_to_token))])
    
    _,topk = torch.topk(cos, k=k+1,)
    for i in topk[1:]:# Remove the input words
        print('cosine sim=%.3f: %s' % (cos[i], (idx_to_token[i])))

get_similar_tokens('chip', 3, net.central_emb.weight.data)

cosine sim=0.516: intel
cosine sim=0.438: screens
cosine sim=0.430: microprocessor


In [207]:
import pickle

In [209]:
with open('embeddings', 'wb') as embeddings_obj:
    pickle.dump(net.central_emb, embeddings_obj)

In [213]:
embs = [net.central_emb(torch.LongTensor([i])).tolist() for i in range(len(idx_to_token))]

In [215]:
%store embs

Stored 'embs' (list)


In [217]:
%store token_to_idx
%store idx_to_token

Stored 'token_to_idx' (dict)
Stored 'idx_to_token' (list)
